In [63]:
import os
import sys
import argparse
import json
import warnings
import logging
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, load_from_disk
import transformers
from peft import PeftModel
from colorama import Fore, Style

from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    GenerationConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training
)

## 定义辅助函数

在微调过程中，我们需要一些辅助函数来处理数据和评估模型。

### 数据预处理函数

In [24]:
def generate_training_data(data_point):
    """
    将输入和输出文本转换为模型可读取的 tokens。

    参数：
    - data_point: 包含 "instruction"、"input" 和 "output" 字段的字典。

    返回：
    - 包含模型输入 IDs、标签和注意力掩码的字典。
    
    示例:
    - 如果你构建了一个字典 data_point_1，并包含字段 "instruction"、"input" 和 "output"，你可以像这样使用函数：
        generate_training_data(data_point_1)
    """
    # 构建完整的输入提示词
    prompt = f"""\
[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem. 你是一個樂於助人的助手且擅長寫唐詩。
<</SYS>>

{data_point["instruction"]}
{data_point["input"]}
[/INST]"""

    # 计算用户提示词的 token 数量
    len_user_prompt_tokens = (
        len(
            tokenizer(
                prompt,
                truncation=True,
                max_length=CUTOFF_LEN + 1,
                padding="max_length",
            )["input_ids"]
        ) - 1
    )

    # 将完整的输入和输出转换为 tokens
    full_tokens = tokenizer(
        prompt + " " + data_point["output"] + "</s>",
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )["input_ids"][:-1]

    return {
        "input_ids": full_tokens,
        "labels": [-100] * len_user_prompt_tokens + full_tokens[len_user_prompt_tokens:],
        "attention_mask": [1] * len(full_tokens),
    }

### 模型评估函数

In [25]:
def evaluate(instruction, generation_config, max_len, input_text="", verbose=True):
    """
    使用 Qwen 格式生成响应。
    """
    prompt = (
        "<|im_start|>system\n你是一位擅長寫唐詩的中文助手。\n<|im_end|>\n"
        f"<|im_start|>user\n{instruction}\n{input_text}\n<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]

    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            max_new_tokens=max_len,
            return_dict_in_generate=True,
            output_scores=True
        )

    output = tokenizer.decode(generation_output.sequences[0], skip_special_tokens=False)
    # 清洗输出：截断 assistant 开头后面的内容
    if "<|im_start|>assistant" in output:
        output = output.split("<|im_start|>assistant")[1]
    if "<|im_end|>" in output:
        output = output.split("<|im_end|>")[0]
    output = output.strip()

    if verbose:
        print(output)
    return output

In [26]:
""" 你可以（但不一定需要）更改 LLM 模型 """

model_name = "Qwen/Qwen2-1.5B-Instruct"

# model_name = "/content/TAIDE-LX-7B-Chat"
# 如果你想使用 TAIDE 模型，请先查看 TAIDE L Models Community License Agreement (https://drive.google.com/file/d/1FcUZjbUH6jr4xoCyAronN_slLgcdhEUd/view)。
# 一旦使用，即表示你同意协议条款。
# !wget -O taide_7b.zip "https://www.dropbox.com/scl/fi/harnetdwx2ttq1xt94rin/TAIDE-LX-7B-Chat.zip?rlkey=yzyf5nxztw6farpwyyildx5s3&st=s22mz5ao&dl=0"
# !unzip taide_7b.zip

In [54]:
import torch
import logging
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig
)

# ✅ 设置设备（MPS 优先）
device = torch.device("cpu")
print(f"✅ 使用設備：{device}")

# ✅ 本地模型路径（重点！直接指向模型文件夹）
model_path = "./cache/Qwen2-1.5B-Instruct"  # ✅ 你的模型必须在这个路径下

# ✅ 加载模型
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_path,  # ✅ 强调命名参数，避免误解为 repo id
    low_cpu_mem_usage=True # MPS 仅支持 float32
).to(device)

# ✅ 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_path,
    add_eos_token=True
)
tokenizer.pad_token = tokenizer.eos_token

# ✅ 设置生成参数
generation_config = GenerationConfig(
    do_sample=True,
    temperature=0.1,
    num_beams=1,
    top_p=0.3,
    no_repeat_ngram_size=3,
    pad_token_id=tokenizer.pad_token_id,
)

✅ 使用設備：cpu


In [37]:
print(model.dtype)

torch.float32


In [28]:
""" 样例和 Prompt 都保持繁体 """
max_len = 128
# 测试样例
test_tang_list = [
    '相見時難別亦難，東風無力百花殘。',
    '重帷深下莫愁堂，臥後清宵細細長。',
    '芳辰追逸趣，禁苑信多奇。'
]

# 获取每个样例的模型输出
demo_before_finetune = []
for tang in test_tang_list:
    demo_before_finetune.append(
        f'模型輸入:\n以下是一首唐詩的第一句話，請用你的知識判斷並完成整首詩。{tang}\n\n模型輸出:\n' +
        evaluate('以下是一首唐詩的第一句話，請用你的知識判斷並完成整首詩。', generation_config, max_len, tang, verbose=False)
    )

# 打印并将输出存储到文本文件
for idx in range(len(demo_before_finetune)):
    print(f"Example {idx + 1}:")
    print(demo_before_finetune[idx])
    print("-" * 80)

Example 1:
模型輸入:
以下是一首唐詩的第一句話，請用你的知識判斷並完成整首詩。相見時難別亦難，東風無力百花殘。

模型輸出:
此詩為唐代詩人李商隱的《無题》詩，全詩如下：

相見时难别亦难，东风无力百花残。

春心莫共花争发，一寸相思一寸灰。

此詩描寫了男女之間的難以割舍的情感，以及對別離的無奈和痛苦。首句「相見难」，暗示了男女兩人的相見之難，而「別亦难」則暗示了別離之難。第二句「東風无力百花殘」，描繪了春風无力，百花凋零的景象，
--------------------------------------------------------------------------------
Example 2:
模型輸入:
以下是一首唐詩的第一句話，請用你的知識判斷並完成整首詩。重帷深下莫愁堂，臥後清宵細細長。

模型輸出:
此詩為唐代詩人杜甫所作，原詩為：

重帷深深下莫樓，臵酒清宵長細流。
此詩描繪了一個女子在深夜時分，獨自坐在深處的樓上，與朋友共飲的情景。
--------------------------------------------------------------------------------
Example 3:
模型輸入:
以下是一首唐詩的第一句話，請用你的知識判斷並完成整首詩。芳辰追逸趣，禁苑信多奇。

模型輸出:
此詩為唐代詩人杜甫所作，整首诗如下：

芳辰迎逸趣，
禁苑见奇才。
春色满园关不住，
一枝红杏出墙来。
--------------------------------------------------------------------------------


### 设置用于微调的超参数

In [55]:
import os

num_train_data = 1040  # 训练数据量，按需求调整

output_dir = "./output"
ckpt_dir = "./exp1"
num_epoch = 1
LEARNING_RATE = 3e-4

cache_dir = "./cache"
from_ckpt = False
ckpt_name = None
dataset_dir = "./GenAI-Hw5/Tang_training_data.json"

logging_steps = 20
save_steps = 65
save_total_limit = 3
report_to = "none"

MICRO_BATCH_SIZE = 1  # 适配 MPS，显存有限，调小
BATCH_SIZE = 4  # 如果可以，累积4个微批次
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE

CUTOFF_LEN = 256

LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

VAL_SET_SIZE = 0  # 不拆分验证集，节省计算

# Qwen2-1.5B-Instruct 可能适用的 target_modules，保持简洁
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]

device_map = None  # MPS 设备不支持 device_map，加载后手动 to(device)

world_size = 1
ddp = False

# 设备定义
import torch
device = torch.device("cpu")
print(f"使用设备：{device}")

使用设备：cpu


In [56]:
print("模型参数类型:", next(model.parameters()).dtype)

模型参数类型: torch.float32


### 开始微调

In [64]:
# 设置TOKENIZERS_PARALLELISM为false，这里简单禁用并行性以避免报错
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 创建指定的输出目录
os.makedirs(output_dir, exist_ok=True)
os.makedirs(ckpt_dir, exist_ok=True)

# 根据 from_ckpt 标志，从 checkpoint 加载模型权重
if from_ckpt:
    model = PeftModel.from_pretrained(model, ckpt_name)

# 对量化模型进行预处理以进行训练
model = prepare_model_for_kbit_training(model)

# 使用 LoraConfig 配置 LORA 模型
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

# 将 tokenizer 的填充 token 设置为 0
tokenizer.pad_token_id = 0

# 加载并处理训练数据
with open(dataset_dir, "r", encoding="utf-8") as f:
    data_json = json.load(f)
with open("tmp_dataset.json", "w", encoding="utf-8") as f:
    json.dump(data_json[:num_train_data], f, indent=2, ensure_ascii=False)

data = load_dataset('json', data_files="tmp_dataset.json", download_mode="force_redownload")

# 将训练数据分为训练集和验证集（若 VAL_SET_SIZE 大于 0）
if VAL_SET_SIZE > 0:
    train_val = data["train"].train_test_split(
        test_size=VAL_SET_SIZE, shuffle=True, seed=42
    )
    train_data = train_val["train"].shuffle().map(generate_training_data)
    val_data = train_val["test"].shuffle().map(generate_training_data)
else:
    train_data = data['train'].shuffle().map(generate_training_data)
    val_data = None

# 使用 Transformers Trainer 进行模型训练
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        num_train_epochs=num_epoch,
        learning_rate=LEARNING_RATE,
        logging_steps=logging_steps,
        save_steps=save_steps,
        output_dir=ckpt_dir,
        save_total_limit=save_total_limit,
        ddp_find_unused_parameters=False if ddp else None,  # 是否使用 DDP，控制梯度更新策略
        report_to=report_to,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# 禁用模型的缓存功能
model.config.use_cache = False

# 若使用 PyTorch 2.0 以上版本且非 Windows 系统，编译模型
if torch.__version__ >= "2" and sys.platform != 'win32':
    model = torch.compile(model)

# 开始模型训练
trainer.train()

# 将训练好的模型保存到指定目录
model.save_pretrained(ckpt_dir)

# 打印训练过程中可能出现的缺失权重警告信息
print("\n 如果上方有关于缺少键的警告，请忽略 :)")


Generating train split: 1040 examples [00:00, 70167.07 examples/s]
Map: 100%|██████████| 1040/1040 [00:00<00:00, 4343.62 examples/s]


ValueError: fp16 mixed precision requires a GPU (not 'mps').